In [4]:
from random import choice
from string import ascii_lowercase

def generate_random_string(length=10):
    letters = ascii_lowercase
    return ''.join(choice(letters) for i in range(length))

In [5]:
s3_bucket = "apper-training-sagemaker-deployments"
prefix = generate_random_string()

In [7]:
model_a = f"s3://{s3_bucket}/{prefix}/files/model.xgboost-1.tar.gz"
model_b = f"s3://{s3_bucket}/{prefix}/files/model.xgboost-2.tar.gz"

In [8]:
!aws s3 cp files/model.xgboost-1.tar.gz {model_a}
!aws s3 cp files/model.xgboost-2.tar.gz {model_b}

upload: files/model.xgboost-1.tar.gz to s3://apper-training-sagemaker-deployments/vfeywdsclu/files/model.xgboost-1.tar.gz
upload: files/model.xgboost-2.tar.gz to s3://apper-training-sagemaker-deployments/vfeywdsclu/files/model.xgboost-2.tar.gz


In [9]:
import sagemaker
from sagemaker import get_execution_role

session = sagemaker.Session()
role = get_execution_role()

In [10]:
from sagemaker.image_uris import retrieve

image_uri = retrieve("xgboost", region="us-east-1", version="0.90-2")

In [13]:
model_data_prefix = f"s3://{s3_bucket}/{prefix}/artifacts/"

In [14]:
from sagemaker.multidatamodel import MultiDataModel

multi_model = MultiDataModel(
    name="MultiModel" + generate_random_string(),
    model_data_prefix=model_data_prefix, 
    image_uri=image_uri,
    role=role)

In [15]:
multi_model.add_model(model_a)
multi_model.add_model(model_b)

's3://apper-training-sagemaker-deployments/vfeywdsclu/artifacts/vfeywdsclu/files/model.xgboost-2.tar.gz'

In [17]:
list(multi_model.list_models())

['vfeywdsclu/files/model.xgboost-1.tar.gz',
 'vfeywdsclu/files/model.xgboost-2.tar.gz']

In [18]:
endpoint_name = "multi-model-endpoint-" + prefix

multi_model.deploy(
    initial_instance_count=1, 
    instance_type='ml.t2.medium', 
    endpoint_name=endpoint_name)

----------------------!

In [19]:
from sagemaker.predictor import Predictor
from sagemaker.serializers import CSVSerializer
from sagemaker.deserializers import JSONDeserializer

In [20]:
predictor = Predictor(endpoint_name=endpoint_name)
predictor.serializer = CSVSerializer()
predictor.deserializer = JSONDeserializer()

In [25]:
a, b = list(multi_model.list_models())

In [26]:
predictor.predict(data="10,-5", target_model=a)

[0.895996630191803]

In [27]:
predictor.predict(data="10,-5", target_model=b)

[0.8308258652687073]

In [28]:
predictor.delete_endpoint()